# MultiScale Classifier

Sections:

1. Data
2. Hyper-parameter space
3. HPO
4. Cross-validation scores


## Setup

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import json
import ray
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

import skexplain
from skexplain.common.importance_utils import to_skexplain_importance

from src.acnets.pipeline import MultiScaleClassifier, Parcellation
from functools import partial

In [ ]:
N_RUNS = 100      # 10 independent train/test runs
TEST_SIZE = .25  # proportion of test subjects out of 32 subjects

## Prepare data

In [ ]:
# Xy
subjects = Parcellation(atlas_name='difumo_64_2mm').fit_transform(None).coords['subject'].values
X = subjects.reshape(-1,1)                                  # subjects ids, shape: (n_subjects, 1)

y_encoder = LabelEncoder()
y = y_encoder.fit_transform([s[:4] for s in subjects])      # labels (AVGP=1 or NVGP=1), shape: (n_subjects,)
y_mapping = dict(zip(y_encoder.classes_, y_encoder.transform(y_encoder.classes_)))

# DEBUG (report label mapping)
print('[DEBUG] label mapping:', y_mapping)

# DEBUG (expected to overfit, i.e., accuracy is 1)
overfit_score = MultiScaleClassifier().fit(X, y).score(X, y)
print(f'[DEBUG] overfit accuracy: {overfit_score:.3f}')

## Hyper-parameter tuning

### Parameter space

In [ ]:
param_space = {
    # 'atlas': ['dosenbach2010', 'gordon2014_2mm', 'difumo_64_2mm'],
    'atlas': tune.choice(['dosenbach2010']),
    'kind': tune.choice(['partial correlation']),
    'extract_h1_features': tune.grid_search([True]),
    'extract_h2_features': tune.grid_search([True]),
    'extract_h3_features': tune.grid_search([True]),
    # 'clf__subsample': tune.choice([.5, .8, 1]),
    'clf__n_estimators': tune.grid_search([100, 200]),
    'clf__max_depth': tune.grid_search([2, 4, 6, 8]),
    'clf__learning_rate': tune.grid_search([.1, .3]),
}


In [ ]:
# now we define the objective function

from ray import train

def eval_multiscale_model(config, classifier, X, y, n_runs=10):

    pipe = MultiScaleClassifier(classifier=classifier).set_params(**config)

    for _ in range(n_runs):
        # outer CV (for test set), and inner CV (for validation set)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, stratify=y)
        inner_cv = StratifiedKFold(n_splits=8, shuffle=True)

        # fit and score the validation set
        val_score = cross_val_score(pipe, X_train, y_train, scoring='accuracy', cv=inner_cv).mean()

        # test score (we only report this and do not use it during HPO)
        test_score = pipe.fit(X_train, y_train).score(X_test, y_test)

        train.report({'val_accuracy': val_score, 'test_score': test_score})

### HPO

In [33]:
# prep the hyper-parameter space and init the objective function
output_name = f'models/multiscale_classifier-xgb-hpo.json'

clf = XGBClassifier(base_score=.5, objective='binary:logistic')
objective_func = partial(eval_multiscale_model, classifier=clf, X=X, y=y, n_runs=5)

ray.shutdown(); ray.init()

tuner = tune.Tuner(
    objective_func,
    param_space=param_space,
    tune_config=tune.TuneConfig(
        metric='val_accuracy',
        mode='max'
    )
)

tuner.fit()
ray.shutdown()

clear_output()
best_result = tuner.get_results().get_best_result(scope='avg')
best_score = best_result.metrics['val_accuracy']
best_params = best_result.config

# store the best hyper-parameters
with open(output_name, 'w') as f:
    best_params['classifier'] = 'XGBClassifier'
    json.dump(best_params, f, indent=2)
    del best_params['classifier']  # TEMP: remove the classifier name

# report best score and best model
print('[DEBUG] Best HPO validation score:', best_score)
MultiScaleClassifier(classifier=clf).set_params(**best_params)

## Cross-validation accuracy and CI

In [ ]:
clf = XGBClassifier(base_score=.5, objective='binary:logistic')
tuned_pipeline = MultiScaleClassifier(classifier=clf).set_params(**best_params)

cv_scores = cross_val_score(tuned_pipeline, X, y,
                            cv=StratifiedShuffleSplit(n_splits=N_RUNS, test_size=TEST_SIZE),
                            verbose=3, n_jobs=-1)

# Calculate 95% confidence interval
bootstrap_ci = stats.bootstrap(cv_scores.reshape(1,-1), np.mean)

# Report
clear_output(wait=True)
print(f'Test accuracy (mean ± std): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}')
print(bootstrap_ci.confidence_interval)